In [ ]:
import numpy as np

import os
os.environ['ARRAY_MODULE'] = 'numpy'
from asl_bloch_sim import animation

In [ ]:
start = 0.26 # 0.7 # seconds
end = 0.32 # 0.808 # seconds

In [ ]:
data = np.load('casl_simulation.npz')
time = data['time']

ts = time[np.argmin(abs(time - start)):np.argmin(abs(time - end))]
mags = data['mags'][np.argmin(abs(time - start)):np.argmin(abs(time - end))]
B_field = data['B_field'][np.argmin(abs(time - start)):np.argmin(abs(time - end))]

In [ ]:
import scipy.signal as sig
import bokeh.plotting as bkp
import bokeh.models as bkm
bkp.output_notebook()

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Magnetization')
plot.line(ts, mags[..., 0], line_width=2, legend_label='Mx', alpha=0.5)
plot.line(ts, mags[..., 1], line_width=2, legend_label='My', color='orange', alpha=0.5)
plot.line(ts, mags[..., 2], line_width=2, legend_label='Mz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.x_range = bkm.DataRange1d(start=ts[0], end=ts[-1])
plot.legend.click_policy = 'hide'

bkp.show(plot)

In [ ]:
NFFT = 2 ** 13
freq = np.fft.fftshift(np.fft.fftfreq(NFFT, ts[1] - ts[0]))
amp = np.log10(np.abs(np.fft.fftshift(np.fft.fft(mags, n=NFFT, axis=0)))) * 20
# plot RF with bokeh
plot = bkp.figure(width=800, height=400, title='Magnetization FFT')
plot.line(freq, amp[..., 2], line_width=2)
plot.xaxis.axis_label = 'Frequency (Hz)'
plot.yaxis.axis_label = 'RF Amplitude (dB)'
bkp.show(plot)

In [ ]:
def rescale(arr, ogtime, newtime):
    # return sig.resample(arr, newtime.size, axis=0, window='rect')
    return sig.resample_poly(arr, 1, ogtime.size // newtime.size, axis=0, padtype='line')
    # return arr[::ogtime.size // newtime.size]

In [ ]:
# anim_duration = 18.75 # s
time_increment = 0.05 # s

anim_time, anim_m = animation.downsample(mags, 0.05, '~20')
anim_time, anim_b = animation.downsample(B_field, 0.05, '~20')

# = np.arange(0, anim_duration, 1/sample_rate)
# anim_m = rescale(mags, ts, anim_time)
# anim_b = rescale(B_field, ts, anim_time)
# ts.size / anim_time.size

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Beff')
plot.line(anim_time, anim_b[..., 0], line_width=2, legend_label='Bx', alpha=0.5)
plot.line(anim_time, anim_b[..., 1], line_width=2, legend_label='By', color='orange', alpha=0.5)
plot.line(anim_time, anim_b[..., 2], line_width=2, legend_label='Bz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'B (µT)'
plot.x_range = bkm.DataRange1d(start=anim_time[0], end=anim_time[-1])
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Magnetization')
plot.line(anim_time, anim_m[..., 0], line_width=2, legend_label='Mx', alpha=0.5)
plot.line(anim_time, anim_m[..., 1], line_width=2, legend_label='My', color='orange', alpha=0.5)
plot.line(anim_time, anim_m[..., 2], line_width=2, legend_label='Mz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.x_range = bkm.DataRange1d(start=0, end=anim_time[-1])
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
animation.speed(ts, anim_time)

In [ ]:
animation.bloch_sphere(anim_m, anim_b, np.gradient(anim_time), animation.speed(ts, anim_time),
                       quality='high_quality')